In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

# Reading

In [ ]:
df = pd.read_csv('cleared_df.csv')
df = df[['date','product_id','store_id','sale_amount']]
df.date = pd.to_datetime(df.date)
df = df.sort_values('date')

# Generate dataset for pipeline

In [ ]:
def get_days(X, begin_date):
    date = begin_date
    product_id = X.iloc[0].product_id
    store_id = X.iloc[0].store_id
    sales = [ 0 for i in range(14)]
    for i in range(14):
        day = X[X.date == date + timedelta(days = i) ]
        if len(day)==0:
            sales[i] = 0.0
        else:
            sales[i] = day.sale_amount.iloc[0]

    return pd.Series([date, product_id, store_id, sales],index=['date', 'product_id', 'store_id', 'sales'])
    
start_date = df.sort_values('date').date.iloc[0]

series_list = []  
targets = []
for i in range(365*3//7):
    print("Progress : " + str( round(100*i/(365*3//7),2)) + "%")
    X_begin = start_date + timedelta(days=i*7)
    X_end = start_date + timedelta(days=(i+2)*7)
    y_begin = start_date + timedelta(days=(i+3)*7)
    y_end = start_date + timedelta(days=(i+4)*7)
    full_period = df[(df.date >= X_begin) & (df.date < y_end)]
    X_period = full_period[full_period.date < X_end]
    y_period = full_period[full_period.date >= y_begin]
    stores_products = full_period.groupby(['store_id', 'product_id']).max().index
    for store, product in stores_products:
        #print(store, product)
        sp_X_period = X_period[(X_period.product_id == product) & (
            X_period.store_id == store)]
        sp_y_period = y_period[(y_period.product_id == product) & (
            y_period.store_id == store)]
        if len(sp_X_period) != 0 and len(sp_y_period) != 0:
            series_list.append(get_days(sp_X_period, X_begin).append(pd.Series([sp_y_period.sale_amount.sum()], index = ['target'])))




In [ ]:
concated = pd.DataFrame( series_list)
concated.to_csv('Ядерный_чемоданчик_clear.csv')

In [ ]:
store_df = df.groupby('store_id').max()[['store_address','region','type']]
store_df.to_csv('stores.csv')

In [ ]:
product_df =df.groupby('product_id').first()[['product_name','product_type','manufacturer']]
product_df.to_csv('products.csv')